# Synthetized beam on the sky

In [ ]:
import glob
import os 

import numpy as np
import healpy as hp

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

%matplotlib inline
%matplotlib notebook

from matplotlib import rc
rc('figure',figsize=(15,15))
rc('font',size=14)
#rc('text',usetex=False)

import qubic 
import qubic.sb_fitting as sbfit
import qubic.selfcal_lib as scal

from qubicpack.pixel_translation import make_id_focalplane, tes2index

In [ ]:
# Dictionary
d = qubic.qubicdict.qubicDict()
d.read_from_file('pipeline_demo.dict')

# d.read_from_file(dictfilename)
d['config'] = 'TD'

d['synthbeam_kmax'] = 3

# Instrument and scene
q = qubic.QubicInstrument(d)
s = qubic.QubicScene(d)

### Synthetic beams on the sky for all TES

In [ ]:
sb = q.get_synthbeam(s, idet=None, external_A=None, hwp_position=0)
print(sb.shape)

#### Coordinates of the peaks (spherical coordinates in radian) for each TES

In [ ]:
kmax = d['synthbeam_kmax']
npeaks = (2 * kmax + 1)**2

horn_spacing = q.horn.spacing
horn_angle = q.horn.angle
nu = d['filter_nu']

position_TES = q.detector.center

# Coordinate on a simple grid
theta, phi = q._peak_angles_kmax(kmax, horn_spacing, horn_angle, nu, position_TES)
print(theta[0])

# Coordinates ordered from highest intensity to the smallest one
theta_order, phi_order, val = q._peak_angles(s, nu, q.detector.center, q.synthbeam, q.horn, q.primary_beam)


In [ ]:
# Plot
TES = 2

# plt.figure(figsize=(8, 8))
# plt.axis('off')
hp.gnomview(sb[TES], min=0, max=1e5, rot=(0, 90), reso=30, title='Grid numbering')
for p in range(npeaks):
    th = theta[TES, p]
    ph = phi[TES, p]
#     hp.visufunc.projscatter(th, ph, color='w', marker='+')
    hp.visufunc.projtext(th-0.02, ph-0.02, str(p), color='w', fontsize=14)

# plt.figure(figsize=(8, 8))
# plt.axis('off')
hp.gnomview(sb[TES], min=0, max=1e5, rot=(0, 90), reso=30, title='Intensity numbering')
for p in range(npeaks):
    th2 = theta_order[TES, p]
    ph2 = phi_order[TES, p]
#     hp.visufunc.projscatter(th2, ph2, color='w', marker='+')
    hp.visufunc.projtext(th2+0.02, ph2+0.02, str(p), color='w', fontsize=14)

#### Print on the sky the motion of the one peak for all TES 

In [ ]:
# Choose a peak
p = 30
hp.gnomview(sb[200] * 0., min=0, max=1e6, rot=(0, 90), reso=12, title=None)
for TES in range(248):
    color = 'w'
    th = theta[TES, p]
    ph = phi[TES, p]
    hp.visufunc.projscatter(th, ph, color=color, marker='.')
#     hp.visufunc.projtext(th, ph, str(TES), color=color, fontsize=10)


### Coordinates of the TES in ONAFP

In [ ]:
xONAFP, yONAFP, vONAFP = scal.get_TEScoordinates_ONAFP(q)
print(xONAFP.shape)

### Plot

To compare simu/measurement, it looks we have to do the following (not sure):
* Measurement: centered on (lon=0, lat=0, phi=0)
* Simulation: centered on (lon=0, lat=90, phi=180)

In [ ]:
# Choose a TES (Instrument numerotation)
TES, ASIC = 6, 1
x0, y0, FP_index, index_q = scal.TES_Instru2coord(TES, ASIC, q, frame='ONAFP')

plt.subplots(1, 2, figsize=(12, 7))
plt.suptitle(f'TES {TES} - ASIC {ASIC} - Index_q {index_q}')

# Focal plane
plt.subplot(121)
plt.scatter(xONAFP, yONAFP, marker='s', s=100, alpha=0.3)
plt.xlabel('X_ONAFP')
plt.ylabel('Y_ONAFP')
plt.plot(x0, y0, 'ro')
plt.axis('square')
plt.title('Focal plane')

# Simulation
plt.subplot(122)
plt.axis('off')
hp.gnomview(sb[index_q], min=0, max=1e5, rot=(0, 90, 180), reso=12, sub=(122),
            title='Simulation', cbar=False)
hp.graticule()
# for p in range(npeaks):
#     th = theta[index_q, p]
#     ph = phi[index_q, p]
#     th2 = theta_order[index_q, p]
#     ph2 = phi_order[index_q, p]
#     hp.visufunc.projtext(th-0.02, ph-0.02, str(p), color='w', fontsize=10) 
#     hp.visufunc.projtext(th2+0.02, ph2+0.02, str(p), color='r', fontsize=10) 



### Animation to see the beam moving

In [ ]:
def animate(det):
    index = q.detector.index[det]
       
    # Focal plane
#     ax1.annotate(str(tes_index[tes]),  xy=(x, y), fontsize=10, color='r')
    
    x = xONAFP[det]
    y = yONAFP[det]
    point.set_data(x, y)
#     ax1.set_title('TES {}'.format(index))
    
    
    # Simulation
    hp.gnomview(sb[det], min=0, max=1e5, rot=(0, 90, 180), reso=20, sub=(122), 
                title='Simulation', cbar=False)
    hp.graticule()
    
    # pixel with Highest intensity
    nside = hp.get_nside(sb[det])
    pix_max = np.argmax(sb[det])
    lon, lat = hp.pix2ang(hp.get_nside(sb[det]), pix_max, lonlat=True)
    hp.visufunc.projscatter(lon, lat, color='r', marker='+', s=500, lonlat=True, label='Highest pixel')
    plt.legend()
    # Numbering of the peaks (absolute and ordered by intensity)
#     for p in range(npeaks):
#         th = theta[det, p]
#         ph = phi[det, p]
#         th2 = theta_order[det, p]
#         ph2 = phi_order[det, p]
# #         hp.visufunc.projscatter(th, ph, color='w', marker='+')
#         hp.visufunc.projtext(th-0.02, ph-0.02, str(p), color='w', fontsize=12) 
#         hp.visufunc.projtext(th2+0.02, ph2+0.02, str(p), color='r', fontsize=12) 
    

def init():
#     ax1.set_title('Synthetic beam on the sky')
    ax1.axis('square')
    ax2.axis('off')

    
fig, axs = plt.subplots(1, 2, figsize=(12, 8))
ax1, ax2 = np.ravel(axs)
ax1.scatter(xONAFP, yONAFP, marker='s', s=200, alpha=0.3)
ax1.set_xlabel('X_ONAFP')
ax1.set_ylabel('Y_ONAFP')

det0 = 238
x0 = xONAFP[det0]
y0 = yONAFP[det0]
point, = ax1.plot(x0, y0, 'ro')
steps = np.arange(det0, det0+10, 1)

anim = FuncAnimation(fig, animate, steps, init_func=init, interval=1000, blit=True, repeat=False)

# Save a .gif
# anim.save('./animation_beam.gif', writer='imagemagick')